<a href="https://colab.research.google.com/github/josephjz/Biomedical_Image_Analysis_Final/blob/main/BIAanalysis_Jennifer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Uploading Data

In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
import os
import pandas as pd
from tqdm import tqdm
import numpy as np
import re

#Pre-processing

In [3]:
#Putting all the notes into an array where the days are the rows and the columns are the rats 
#the value at each position is the delay for that day (row) and rat (col)
#in the format of day4 rat 1: 0:08-5:08
reference = np.zeros((6,17))
with open('/content/drive/MyDrive/BIA_folder/preproctimes.txt') as time_file:
    for line in time_file:
      nums = re.findall(r'\d+', line) #finding all the numbers in the string)
      time = re.findall(r'\d+',re.findall(r'\d+-',line)[0])[0] #getting the time
      #print('day: ', nums[0])
      #print('rat: ', nums[1])
      #print('time: ', time)
      reference[int(nums[0]), int(nums[1])]=int(time)

reference[4,16]='08'
#reference

In [4]:
#Get video CSVs into list
framespersecond=30
file_names = [] #only getting csv files
csv_list = []
count = 0
right_left = []
for file_name in tqdm(os.listdir('/content/drive/MyDrive/BIA_folder/csvs/')): #iterating through list
  if file_name.endswith('.csv'): #only getting csv files
    file_names.append(file_name)
    csv = pd.read_csv('/content/drive/MyDrive/BIA_folder/csvs/'+file_name, dtype='a')
    numbers = re.findall(r'\d+', file_name)
    start_time = int(reference[int(numbers[1]),int(numbers[2])])
    csv.drop(csv.index[2:start_time*framespersecond],axis=0, inplace=True)
    csv.drop(csv.index[start_time*framespersecond+1:start_time*framespersecond+9001],axis=0, inplace=True)
    csv_list.append(csv.to_numpy())
    if '(1)' in file_name:
      right_left.append('R')
      count+=1
    else: 
      right_left.append('L')

100%|██████████| 129/129 [00:17<00:00,  7.56it/s]


In [5]:
print(csv_list[0][0:2])
print(file_names[3]) # using this for testing freezing bc i have that video locally 

print(file_names.index('AD2205_Day2_Rat_1DLC_resnet50_BIAfinal22Apr11shuffle1_500000.csv'))

[['bodyparts' 'nose' 'nose' 'nose' 'L eye' 'L eye' 'L eye' 'R eye'
  'R eye' 'R eye' 'L ear' 'L ear' 'L ear' 'R ear' 'R ear' 'R ear'
  'tail base' 'tail base' 'tail base']
 ['coords' 'x' 'y' 'likelihood' 'x' 'y' 'likelihood' 'x' 'y' 'likelihood'
  'x' 'y' 'likelihood' 'x' 'y' 'likelihood' 'x' 'y' 'likelihood']]
AD2205_Day3_Rat_13DLC_resnet50_BIAfinal22Apr11shuffle1_500000.csv
49


In [6]:
for csv in csv_list: 
  #print(csv)
  #convert items from object to ints
  for row in range(2,csv.shape[0]):
    for col in range(1,csv.shape[1]):
      csv[row,col] = float(csv[row,col])

#Error Rate

In [7]:
print(len(csv_list)) 
csv_list[0]

128


array([['bodyparts', 'nose', 'nose', ..., 'tail base', 'tail base',
        'tail base'],
       ['coords', 'x', 'y', ..., 'x', 'y', 'likelihood'],
       ['358', 651.8460083007812, 356.6088562011719, ...,
        716.3576049804688, 130.51812744140625, 0.44299402832984924],
       ...,
       ['714', 610.7783203125, 374.6498718261719, ..., 783.7279052734375,
        162.2873992919922, 0.9999819993972778],
       ['715', 652.5914306640625, 411.4912109375, ..., 782.6762084960938,
        159.27865600585938, 0.9999964237213135],
       ['716', 651.1100463867188, 405.0489807128906, ...,
        779.6741333007812, 156.10743713378906, 0.9999935626983643]],
      dtype=object)

In [8]:
#calculate the per bodypart sum
import numpy as np
import matplotlib.pyplot as plt

setup = {'nose':[0],'L eye':[0],'R eye':[0], 'L ear':[0],'R ear':[0],'tail base':[0]}

errordf = pd.DataFrame(setup)

histogram = pd.DataFrame(setup)

for index, csv in enumerate(csv_list):
  likelihoods = csv[:, [3,6,9,12,15,18]]
  for ind, bodypart in enumerate(likelihoods[0]):
    avg = np.sum(likelihoods[2:,ind])
    errordf.loc[0,bodypart]+= avg
    #histogram.loc[]
  break
plt.show()
total = likelihoods.shape[0]*len(csv_list)

In [9]:
#divide by total
display(errordf/total)

,nose,L eye,R eye,L ear,R ear,tail base
0,0.005649,0.005872,0.005793,0.007231,0.007175,0.006134


# Variables for Low Probability Correciton and Freezing


In [10]:
csv = pd.read_csv('/content/drive/MyDrive/BIA_folder/csvs/AD2205_Day2_Rat_1DLC_resnet50_BIAfinal22Apr11shuffle1_500000.csv', dtype='a').to_numpy()

column_names = ['nose_x', 'nose_y', 'nose_likelihood',
                'l_eye_x', 'l_eye_y', 'l_eye_likelihood',
                'r_eye_x', 'r_eye_y', 'r_eye_likelihood',
                'l_ear_x', 'l_ear_y', 'l_ear_likelihood',
                'r_ear_x', 'r_ear_y', 'r_ear_likelihood',
                'tail_base_x', 'tail_base_y', 'tail_base_likelihood']


#convert items from object to ints
for row in range(2,csv.shape[0]):
  for col in range(1,csv.shape[1]):
    csv[row,col] = float(csv[row,col])

df_csv = pd.DataFrame(csv[2:,1:], columns = column_names) 

display(df_csv)

# print(df_csv['nose_likelihood'].mean())


,nose_x,nose_y,nose_likelihood,l_eye_x,l_eye_y,l_eye_likelihood,r_eye_x,r_eye_y,r_eye_likelihood,l_ear_x,l_ear_y,l_ear_likelihood,r_ear_x,r_ear_y,r_ear_likelihood,tail_base_x,tail_base_y,tail_base_likelihood
0,353.596649,164.722839,0.999983,317.558136,140.07373,0.999999,321.23764,174.695328,1.0,280.01413,129.969269,0.999998,296.483398,183.200867,1.0,290.394409,322.10733,0.000536
1,352.076202,166.077255,0.999113,319.342499,142.332916,1.0,321.85498,177.030655,1.0,282.109283,130.720093,0.999999,296.392029,184.132141,1.0,291.293549,321.148895,0.002453
2,352.809418,172.543396,0.999959,318.482574,148.094254,1.0,319.14328,181.987808,1.0,279.570099,133.959076,1.0,292.630859,185.409378,1.0,291.35788,321.412628,0.014242
3,347.511963,175.567322,0.999997,320.527466,153.600281,1.0,317.533936,182.883118,1.0,283.616302,132.630432,1.0,291.041107,186.218887,1.0,290.415009,322.674377,0.003761
4,348.40506,178.302216,0.999714,323.280701,158.149734,1.0,315.942169,188.082352,0.999964,289.668152,133.018845,1.0,289.894958,186.214264,1.0,290.35849,322.923859,0.017051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9341,334.340454,332.934723,0.999999,336.785461,299.851501,0.992998,301.006866,307.173248,1.0,329.472839,251.993958,1.0,275.890564,269.243774,1.0,271.294739,68.172798,0.999992
9342,330.423859,334.945251,0.999997,336.322327,301.217102,0.967916,300.083771,307.564178,1.0,327.933868,251.686157,1.0,275.140533,270.016571,1.0,272.30365,67.468468,0.999713
9343,337.121155,332.438843,1.0,340.081909,294.738464,0.99619,305.39209,306.346741,1.0,332.865204,251.60968,1.0,279.406097,268.59549,1.0,273.63504,65.220215,0.996964
9344,339.325958,334.779358,1.0,341.843658,299.785217,0.98683,308.31192,306.896759,1.0,336.44223,252.422775,1.0,282.642487,268.237183,1.0,276.711426,66.539772,0.997054


In [39]:
# test 
print(df_csv['tail_base_likelihood'][923:930].to_numpy())

print(df_csv['tail_base_x'][923:930].to_numpy())

print(df_csv['tail_base_y'][923:930].to_numpy())

x = replace_by_bp_likelihood(df_csv, 10, column_names)

[0.9865080118179321 0.8125309944152832 0.18080651760101318
 0.015712082386016846 0.06705260276794434 0.6628450155258179
 0.8418127298355103]
[359.19439697265625 359.05670166015625 342.2000325520833
 325.34336344401044 308.4866943359375 308.4866943359375 363.54364013671875]
[210.50338745117188 82.1769790649414 133.7161610921224 185.2553431193034
 236.79452514648438 236.79452514648438 81.3436279296875]


In [40]:
print(x['tail_base_likelihood'][923:930].to_numpy())

print(x['tail_base_x'][923:930].to_numpy())

print(x['tail_base_y'][923:930].to_numpy())

[0.9865080118179321 0.8125309944152832 0.18080651760101318
 0.015712082386016846 0.06705260276794434 0.6628450155258179
 0.8418127298355103]
[359.19439697265625 359.05670166015625 346.41419982910156
 333.7716979980469 321.1291961669922 308.4866943359375 363.54364013671875]
[210.50338745117188 82.1769790649414 120.83136558532715 159.4857521057129
 198.14013862609863 236.79452514648438 81.3436279296875]


# Low Probability Correction

In [38]:
def replace_by_bp_likelihood(input_csv_df, pixel_treshold, column_names):

  returned_csv_df = input_csv_df   # 9346 frames in this csv

  for c in range(0,len(column_names),3): # for each body part
    
    good_probability_index = -1 # to keep track of most recent xy cord with high likelihood
    frame = 0 # to iterate through frames 

    x = input_csv_df[column_names[c]].to_numpy() # x column for body part 
    y = input_csv_df[column_names[c+1]].to_numpy() # y column 
    likelihood = input_csv_df[column_names[c+2]].to_numpy() # likelihood 
    
    while frame < len(input_csv_df) - 1: # for each frame

      if likelihood[frame] >= likelihood.mean(): # if the likelihood of that frame's prediction is less than the average for that body part
        good_probability_index = frame # save this frame index as a frame with high prediction likelihood 
        frame += 1 # iterate 

      else: # if the frame has less than the average likelihood for that body part 
        bad_probability_index = [] # start a list to hold these indices 

        while likelihood[frame] < likelihood.mean(): 
          bad_probability_index.append(frame) # add the indices of the frames with below bp average likelihood to list 
          frame += 1

        # once outside this loop, there will be a list of the frames with too low of likelihoods, so we want to replace the values 
        # with the average between the last frame with a likelihood over the bp average, and the next frame w that 

        if bad_probability_index[0] == 0: # if the first frame has too low of a prediction, 
                    # we will replace these first bad frames with the value of the first good frame in the column 
          for i in range(len(bad_probability_index)):
            returned_csv_df[column_names[c]][bad_probability_index[i]] = returned_csv_df[column_names[c]][bad_probability_index[-1] + 1] # x cord
            returned_csv_df[column_names[c+1]][bad_probability_index[i]] = returned_csv_df[column_names[c+1]][bad_probability_index[-1] + 1] # y cord 
        
        elif bad_probability_index[-1] == (len(input_csv_df) - 1): # if the last frame has too low a prediction, 
                    # we will replace these last bad frames with the value of the last good frame in the column 
          for i in range(len(bad_probability_index)):
            returned_csv_df[column_names[c]][bad_probability_index[i]] = returned_csv_df[column_names[c]][good_probability_index] # x cord
            returned_csv_df[column_names[c+1]][bad_probability_index[i]] = returned_csv_df[column_names[c+1]][good_probability_index] # y cord
                            
        else: # somewhere in the middle, average the high probs next to the low probs  
          
          # first idea was to set all of them to the average, but that will induce incorrect feezing detection 
          # new_x = (x[good_probability_index] + x[bad_probability_index[-1] + 1 ]) / len(bad_probability_index)
          # new_y = (y[good_probability_index] + y[bad_probability_index[-1] + 1 ]) / len(bad_probability_index) 

          # next is to make line between the two points, and increment evenly in between 
          start_coord = [x[good_probability_index], y[good_probability_index]]
          finish_coord = [x[bad_probability_index[-1] + 1 ], y[bad_probability_index[-1] + 1 ]]

          x_difference = finish_coord[0] - start_coord[0] # this way it keeps the direciton too 
          y_difference = finish_coord[1] - start_coord[1] # this way it keeps the direciton too 

          increment_x = x_difference / (len(bad_probability_index) + 1)
          increment_y = y_difference / (len(bad_probability_index) + 1)

          for i in range(len(bad_probability_index)): 
            # first idea was to set all of them to the average, but that will induce incorrect feezing detection 
            # returned_csv_df[column_names[c]][bad_probability_index[i]] = new_x
            # returned_csv_df[column_names[c+1]][bad_probability_index[i]] = new_y

            returned_csv_df[column_names[c]][bad_probability_index[i]] = start_coord[0] + (i+1)*increment_x 
            returned_csv_df[column_names[c+1]][bad_probability_index[i]] = start_coord[1] + (i+1)*increment_y  

  return returned_csv_df

In [32]:
p1 = [10,12]
p2 = [5,7]
distance_x = ((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)**0.5
print(distance_x)

7.0710678118654755


#Freezing versus Motion

## Nose Freeze DUPLICATE THIS


In [59]:
def nose_freezing(input_csv, pixel_threshold, column_names):

  differences = [] # raw frame to frame xy cord distance
  prev_cord = [0,0]
  freeze_detected_in_frame = [] # 0 and 1s for no and yes 

  df_csv = pd.DataFrame(input_csv, columns = column_names) 

  for frame, data in df_csv.iterrows():
    nose_cord = [data.nose_x, data.nose_y]
    if frame == 0:
      prev_cord = nose_cord
      differences.append(0)
      distance = pixel_threshold * 2 # so that freeze will be 0 
    else: 
      distance = ((nose_cord[0] - prev_cord[0])**2 + (nose_cord[1] - prev_cord[1])**2)**0.5
      differences.append(distance)
      prev_cord = nose_cord

    if distance < pixel_threshold:
      freeze_detected_in_frame.append(1)
    else:
      freeze_detected_in_frame.append(0)
    
  return freeze_detected_in_frame

test = nose_freezing(df_csv, 1, column_names)

print(test)
print(sum(test))



[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

## Right Eye Freezing

In [13]:
def r_eye_freezing(input_csv, pixel_treshold, column_names):

  differences = [] # raw frame to frame xy cord distance
  prev_cord = [0,0]
  freeze_detected_in_frame = [] # 0 and 1s for no and yes 

  df_csv = pd.DataFrame(input_csv[2:,1:], columns = column_names) 

  for frame, data in df_csv.iterrows():
    # if data.nose_likelihood < .7:
      # what do we want to do with this?
    r_eye_cord = [data.R_eye_x, data.R_eye_y]
    if frame == 0:
      prev_cord = r_eye_cord
      differences.append(0)
      distance = pixel_threshold * 2 # so that freeze will be 0 
    else: 
      distance = ((r_eye_cord[0] - prev_cord[0])**2 + (r_eye_cord[1] - prev_cord[1])**2)**0.5
      differences.append(distance)
      prev_cord = r_eye_cord
    if distance < pixel_threshold:
      freeze_detected_in_frame.append(1)
    else:
      freeze_detected_in_frame.append(0)

##Left Eye Freezing

In [14]:
def l_eye_freezing(input_csv, pixel_treshold, column_names):

  differences = [] # raw frame to frame xy cord distance
  prev_cord = [0,0]
  freeze_detected_in_frame = [] # 0 and 1s for no and yes 

  df_csv = pd.DataFrame(input_csv[2:,1:], columns = column_names) 

  for frame, data in df_csv.iterrows():
    # if data.nose_likelihood < .7:
      # what do we want to do with this?
    l_eye_coord = [data.L_eye_x, data.L_eye_r]
    if frame == 0:
      prev_cord = l_eye_coord
      differences.append(0)
      distance = pixel_threshold * 2 # so that freeze will be 0 
    else: 
      distance = ((l_eye_coord[0] - prev_cord[0])**2 + (l_eye_coord[1] - prev_cord[1])**2)**0.5
      differences.append(distance)
      prev_cord = l_eye_coord
    if distance < pixel_threshold:
      freeze_detected_in_frame.append(1)
    else:
      freeze_detected_in_frame.append(0)


## Right Ear Freeze

In [15]:
def r_ear_freezing(input_csv, pixel_treshold, column_names):

  differences = [] # raw frame to frame xy cord distance
  prev_cord = [0,0]
  freeze_detected_in_frame = [] # 0 and 1s for no and yes 

  df_csv = pd.DataFrame(input_csv[2:,1:], columns = column_names) 

  for frame, data in df_csv.iterrows():
    # if data.nose_likelihood < .7:
      # what do we want to do with this?
    r_ear_coord = [data.R_ear_x, data.R_ear_y]
    if frame == 0:
      prev_cord = r_ear_coord
      differences.append(0)
      distance = pixel_threshold * 2 # so that freeze will be 0 
    else: 
      distance = ((r_ear_coord[0] - prev_cord[0])**2 + (r_ear_coord[1] - prev_cord[1])**2)**0.5
      differences.append(distance)
      prev_cord = r_ear_coord
    if distance < pixel_threshold:
      freeze_detected_in_frame.append(1)
    else:
      freeze_detected_in_frame.append(0)

## Left Ear Freeze




In [16]:
def l_ear_freezing(input_csv, pixel_treshold, column_names):

  differences = [] # raw frame to frame xy cord distance
  prev_cord = [0,0]
  freeze_detected_in_frame = [] # 0 and 1s for no and yes 

  df_csv = pd.DataFrame(input_csv[2:,1:], columns = column_names) 

  for frame, data in df_csv.iterrows():
    # if data.nose_likelihood < .7:
      # what do we want to do with this?
    l_ear_cord = [data.L_ear_x, data.L_ear_y]
    if frame == 0:
      prev_cord = l_ear_cord
      differences.append(0)
      distance = pixel_threshold * 2 # so that freeze will be 0 
    else: 
      distance = ((l_ear_cord[0] - prev_cord[0])**2 + (l_ear_cord[1] - prev_cord[1])**2)**0.5
      differences.append(distance)
      prev_cord = l_ear_cord
    if distance < pixel_threshold:
      freeze_detected_in_frame.append(1)
    else:
      freeze_detected_in_frame.append(0)


## Tail Base Freeze

In [17]:
def tail_base_freezing(input_csv, pixel_treshold, column_names):

  differences = [] # raw frame to frame xy cord distance
  prev_cord = [0,0]
  freeze_detected_in_frame = [] # 0 and 1s for no and yes 

  df_csv = pd.DataFrame(input_csv[2:,1:], columns = column_names) 

  for frame, data in df_csv.iterrows():
    #data.nose_likelihood < .7:
      # what do we want to do with this?
    tail_cord = [data.tail_base_x, data.tail_base_y]
    if frame == 0:
      prev_cord = tail_cord
      differences.append(0)
      distance = pixel_threshold * 2 # so that freeze will be 0 
    else: 
      distance = ((tail_cord[0] - prev_cord[0])**2 + (tail_cord[1] - prev_cord[1])**2)**0.5
      differences.append(distance)
      prev_cord = tail_cord
    if distance < pixel_threshold:
      freeze_detected_in_frame.append(1)
    else:
      freeze_detected_in_frame.append(0)

## Body Freeze


In [61]:
def body_freeze(input_csv, pixel_threshold, column_names, window_of_frames):

  freeze_window = []

  nose = nose_freezing(input_csv, pixel_threshold, column_names)
  #r_eye = r_eye_freezing(input_csv, pixel_threshold, column_names)
  #l_eye = l_eye_freezing(input_csv, pixel_threshold, column_names)
  #r_ear = r_ear_freezing(input_csv, pixel_threshold, column_names)
  #l_ear = l_ear_freezing(input_csv, pixel_threshold, column_names)
  #tail = tail_base_freezing(input_csv, pixel_threshold, column_names)

  for i in range(len(input_csv)):
    if np.all(nose[i:i+window_of_frames]) and np.all(r_eye[i:i+window_of_frames]) and np.all(l_eye[i:i+window_of_frames]) and np.all(r_eye[i:i+window_of_frames])

      freeze.append(True)
#             elif idx > t_req and np.all(dist_thresholds[idx-t_req:idx]):
#                 freezing.append(True)
#             else:
#                 freezing.append(False)
#         else:
#             freezing.append(False)

  
  
body_freeze(df_csv, 1, column_names, 30)



# def nose_freezing(input_csv, pixel_treshold, column_names):

#   differences = [] # raw frame to frame xy cord distance
#   prev_cord = [0,0]
#   freeze_detected_in_frame = [] # 0 and 1s for no and yes 

#   df_csv = pd.DataFrame(input_csv[2:,1:], columns = column_names) 

#   for frame, data in df_csv.iterrows():
#     nose_cord = [data.nose_x, data.nose_y]
#     if frame == 0:
#       prev_cord = nose_cord
#       differences.append(0)
#       distance = pixel_threshold * 2 # so that freeze will be 0 
#     else: 
#       distance = ((nose_cord[0] - prev_cord[0])**2 + (nose_cord[1] - prev_cord[1])**2)**0.5
#       differences.append(distance)
#       prev_cord = nose_cord
#     if distance < pixel_threshold:
#       freeze_detected_in_frame.append(1)
#     else:
#       freeze_detected_in_frame.append(0)

# def calc_freezing(array, pixel_threshold=1, t_req=30):

#     freezing = []
#     dist_thresholds =  array < pixel_threshold

#     for idx, threshold in enumerate(dist_thresholds):
#         if threshold:
#             if np.all(dist_thresholds[idx:idx+t_req]):
#                 freezing.append(True)
#             elif idx > t_req and np.all(dist_thresholds[idx-t_req:idx]):
#                 freezing.append(True)
#             else:
#                 freezing.append(False)
#         else:
#             freezing.append(False)

#     return freezing

# Other, Notes, Tests


In [18]:



params = get_params()

for rat in params.index:
    bounds_points = get_bounds_points(params, rat)
    x, y = determine_midpoints(bounds_points)
    slope, y_int = determine_midline(x, y)
    pose = get_pose_data(rat, params, DURATION)
    pose['nose', 'left'] = get_side(pose.nose[['x', 'y']], slope, y_int)
    pose['nose', 'displacement'] = calc_distance(pose.nose[['x', 'y']])
    pose['nose', 'freezing'] = calc_freezing(pose['nose', 'displacement'])
    pose['head_center', 'left'] = get_side(pose.head_center[['x','y']],
                                          slope, y_int)
    pose['head_center', 'displacement'] = calc_distance(pose.head_center[['x',
                                                                          'y']])
    pose['head_center', 'feezing'] = calc_freezing(pose['head_center',
                                                        'displacement'])
    pose = pose[['nose', 'head_center']]
    pose.rename_axis('frame')
    pose = pose.reset_index().rename({'index':'frame'}, axis=1)
    pose.to_csv('/Data/Nick/NW2202/data/NW2202D/%s.csv' % rat)

FileNotFoundError: ignored

In [ ]:
sum(rolling_freeze) # 94 for 1, 150 for 2 

In [ ]:
freeze_detected_in_frame

frame_window = 3 

rolling_freeze = []

for idx, freeze_bool in enumerate(freeze_detected_in_frame):
    if freeze_bool == 1:
        if np.all(freeze_detected_in_frame[idx:idx+frame_window]):
            rolling_freeze.append(True)
        elif idx > frame_window and np.all(freeze_detected_in_frame[idx-frame_window:idx]):
            rolling_freeze.append(True)
        else:
            rolling_freeze.append(False)
            
    else:
        rolling_freeze.append(False)

In [ ]:
df_csv0.head()

#Head Direction

In [ ]:
#Determining threshold for each bodypart


In [ ]:
#is it cropped left or right
for ind, csv in enumerate(csv_list):
  #order: nose, L eye, R eye, L ear, R ear
  likelihoods = csv[:, [3,6,9,12,15]]
  Xcoords = csv[:,[1,4,7,10,13]]
  Ycoords = csv[:,[2,4,8,11,14]]
  if right_left[ind]=='R': #RIGHT
    print('Right')
    #if the nose liklihood is below a 

  elif right_left[ind]=='L': #LEFT
    print('Left')
#if its the left side
  #potential interaction is when nose is more right than ears (OVER A CERTAIN THRESHOLD)


# if its the right side
  #potential interaction is when nose is more left than ears (OVER A CERTAIN THRESHOLD)

In [ ]:
#is it cropped left or right

#if its the left side
  #potential interaction is when nose is more right than ears (OVER A CERTAIN THRESHOLD)

# if its the right side
  #potential interaction is when nose is more left than ears (OVER A CERTAIN THRESHOLD)

#TO DO 
1. average likelihood per body part 

||**Average Training Pixel Error**|**Average Test Pixel Error**|
|:-----------|:---------|:---------|
|**First Experiment**|1.56|5.39|
|**Second Experiment**|2.09|19.53|